<a href="https://colab.research.google.com/github/urness/CS167Fall2025/blob/main/Day13_Random_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day13
##Random Forests

#### CS167: Machine Learning, Fall 2025


# Warm up Exercise
#### Download the breast-cancer-wisconsin-data.csv file from blackboard and upload it to your Google Drive

  1. build a decision tree
  2. what is the accuracy?
  3. display a confusion matrix


In [ ]:
# Mount your drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#0. import libraries
import sklearn
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

data= pd.read_csv("/content/drive/MyDrive/CS167/datasets/breast-cancer-data.csv")
data.head()

In [ ]:
#split data
target = "diagnosis"
predictors = data.columns.drop(target) #gets all of the columns except the target
train_data, test_data, train_sln, test_sln = train_test_split(data[predictors], data[target], test_size = 0.2, random_state=41)

In [ ]:
### Warm up Exercise
### Your code Here

# Random Forest Code

#In-Class Exercise #1:
Look at [RandomForestClassifer Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

1.   What is the default number of trees?
2.   Using `random_state = 0` as a parameter to `RandomForestClassifier`, How does increasing or decreasing the number of **trees** affect accuracy? (experiment by increasing/decreasing by 25 of the number of default trees)
3.   What is the parameter to change to affect the number of features used?
4.   Using `random_state = 0` as a parameter to `RandomForestClassifier`, and using the default number of trees, How does adjusting the number of **features** affect accuracy? Consider the possibilities of
  - None
  - 'sqrt'
  - 'log2'
  
  
  Which of the above (if any) will improve the accuracy?



In [ ]:
#### a Random Forest Classifier

# construct a random forest object (use random_state=0 as a parameter)

# call .fit(train_data,train_sln)

# generate predictions using the test_data

# print out accuracy and confusion matrix
print("accuracy score: ", metrics.accuracy_score(test_sln,predictions))

vals = data[target].unique() ## possible classification values (M = malignant; B = benign)
conf_mat = metrics.confusion_matrix(test_sln, predictions, labels=vals)
print(pd.DataFrame(conf_mat, index = "True " + vals, columns = "Predicted " + vals))

# Feature Importances

In [ ]:
# It looks like our random forest model achieved pretty good accuracy.
# Now lets check how important each of the features was in the ensemble of models we built.

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#creates a list of numbers the right size to use as the index
#and sorts the list so that the most important feature are first
index = range(len(predictors))
importances = forest.feature_importances_
sorted_indices = np.argsort(importances)

plt.figure(figsize=(8,10)) #making the table a bit bigger so the text is readable
plt.title('Breast Cancer Feature Importances')
plt.barh(range(len(sorted_indices)),importances[sorted_indices],height=0.8) #horizontal bar chart
plt.ylabel('Feature')
plt.yticks(index,predictors) #put the feature names at the y tick marks
plt.xlabel("Random Forest Feature Importance")
plt.show()

# Tuning the Forest

*   How can we tell how many trees to use?
*   What about how many features to include in our trees?

We can tune our random forest to find the best values of model
parameters:



In [ ]:
#This function just loops through a series of n_estimator values, builds a different model
#for each, and then plots their respective accuracies. By making it a function, it's easier
#to try out different ranges of numbers
import matplotlib.pyplot as plt

def tune_number_of_trees(n_estimator_values):
    rf_accuracies = []

    for n in n_estimator_values:

        curr_rf = RandomForestClassifier(n_estimators=n, random_state=0)
        curr_rf.fit(train_data,train_sln)
        curr_predictions = curr_rf.predict(test_data)
        curr_accuracy = metrics.accuracy_score(test_sln,curr_predictions)
        rf_accuracies.append(curr_accuracy)


    plt.suptitle('Random Forest accuracy vs. number of trees',fontsize=18)
    plt.xlabel('# trees')
    plt.ylabel('accuracy')
    plt.plot(n_estimator_values,rf_accuracies,'ro-')
    plt.axis([0,n_estimator_values[-1]+1,.9,1])

    plt.show()

tune_number_of_trees(range(1,31))

It looks like whether we are using small numbers of trees or large ones, the accuracy stays about the same. It appears at least sometimes that Random Forest doesn't take a lot of tuning of the number of trees.

# Tuning Number of Features

In [ ]:
def tune_max_features(max_features_values):
    rf_accuracies = []

    for m in max_features_values:

        curr_rf = RandomForestClassifier(n_estimators=10,max_features=m, random_state=0)
        curr_rf.fit(train_data,train_sln)
        curr_predictions = curr_rf.predict(test_data)
        curr_accuracy = metrics.accuracy_score(test_sln,curr_predictions)
        rf_accuracies.append(curr_accuracy)


    plt.suptitle('Random Forest accuracy vs. max features',fontsize=18)
    plt.xlabel('max features')
    plt.ylabel('accuracy')
    plt.plot(max_features_values,rf_accuracies,'ro-')
    plt.axis([0,max_features_values[-1]+1,.9,1.01])

    plt.show()

tune_max_features(range(1,11))